In [ ]:
import os
import numpy as np

## 5.19 Rotate a 2D Array 

Image rotation is a fundamental operation in computer graphics. Figure 5.4 illustrates the rotation operation on a 2D array representing a bit-map of an image. Specifically, the image is rotated by 90 degrees clockwise.

Write a function that takes as input an n x n 2D array, and rotates the array by 90 degrees clockwise.
Hint: Focus on the boundary elements.

In [ ]:
def rotate_90_clock_np(arr):
    arr = np.array(arr)
    arr_rot = np.zeros(arr.shape)
    for old_row_ind, new_col_ind in enumerate(range(len(arr))[::-1]):
        arr_rot[:, new_col_ind] = arr[old_row_ind, :]
    return arr_rot


def rotate_coords_90_clock(i, j, n):
    i_rot = j
    j_rot = n - 1 - i
    return i_rot, j_rot


def rotate_corners(arr, i_start, j_start):
    n = len(arr)
    i_orig, j_orig = i_start, j_start
    reassign_val = arr[i_orig][j_orig]
    for _ in range(4):
        i_rot, j_rot = rotate_coords_90_clock(i_orig, j_orig, n)
        temp = arr[i_rot][j_rot]
        arr[i_rot][j_rot] = reassign_val
        reassign_val = temp
        i_orig, j_orig = i_rot, j_rot

        
def rotate_square(arr, ind_start):
    i_start = ind_start
    for j_start in range(ind_start, len(arr) - ind_start - 1):
        rotate_corners(arr, i_start, j_start)

        
def rotate_90_clock_inplace(arr):
    num_square = len(arr) / 2
    for ind_start in range(num_square):
        rotate_square(arr, ind_start)

        
def rotate_90_clock(arr):
    n = len(arr)
    arr_rot = [[None] * n for i in range(n)]
    for i in range(n):
        for j in range(n):
            i_rot, j_rot = rotate_coords_90_clock(i, j, n)
            arr_rot[i_rot][j_rot] = arr[i][j]
    return arr_rot


def rotate(arr):
    for i in xrange(len(arr)/2):
#         for j in xrange(len(arr)/2 + len(arr)%2):
        for j in xrange(i, len(arr) - i - 1):
            rotate_corners(arr,i,j)
    return arr

In [ ]:
# arr = [[i] * n for i in range(5)]
# arr = [[1, 2], [3, 4]]
# arr = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]
arr = [[1, 2, 3, 4, 140], [5, 6, 7, 8, 130], [9, 10, 11, 12, 120], [13, 14, 15, 16, 110], [17, 18, 19, 20, 100]]

# arr_rot = rotate_90_clock(arr)

# print(np.array(arr))
# print('')
# print(np.array(arr_rot))

print(np.array(arr))
# rotate_90_clock_inplace(arr)
rotate(arr)
print('')
print(np.array(arr))

## 5.13 Sample Online Data 

This problem is motivated by the disgn of a packet sniffer that provides a uniform sample of packets for a network session.

Design a program that takes as input a size k, and reads packets, coninuously maintaining a uniform random subset of size k of the read packets.

Hint: Suppose you have a procedure which selects k packets from the first n >= k packets as specified. How would you deal with the (n+1)st packet?

In [ ]:
import random

class PacketSampler:
    
    def __init__(self, k):
        self.k = k
        self.n = 0
        self.subset = []
        
    def receive_packet(self, p):
        if len(self.subset) < self.k:
            self.subset.append(p)
        else:
            replace_prob = float(self.k) / (self.n + 1)
            if random.random() < replace_prob:
                replace_ind = random.randint(0, self.k - 1)
                self.subset[replace_ind] = p
        self.n += 1


class PacketSamplerWrong:
    
    def __init__(self, k):
        self.k = k
        self.n = 0
        self.subset = []
        
    def receive_packet(self, p):
        if len(self.subset) < self.k:
            self.subset.append(p)
        else:
            replace_prob = random.random()
            if random.random() < replace_prob:
                replace_ind = random.randint(0, self.k - 1)
                self.subset[replace_ind] = p
        self.n += 1


def get_packet_subset(n, packet_sampler):
    for i in range(n):
        packet_sampler.receive_packet(i)
    return packet_sampler.subset


def get_simulated_distribution(n, k, sampler_name, num_trials):
    if sampler_name == 'right':
        Sampler = PacketSampler
    elif sampler_name == 'wrong':
        Sampler = PacketSamplerWrong
    else:
        raise ValueError('Unrecognized sampler name.')
    
    packet_counter = {p: 0 for p in range(n)}
    for trial_ind in range(num_trials):
        subset = get_packet_subset(n, Sampler(k))
        for p in subset:
            packet_counter[p] += 1
    
    return packet_counter

In [ ]:
num_trials = 500000
n = 10
k = 3
sampler_name = 'wrong'

packet_counts = get_simulated_distribution(n=n, k=k, sampler_name=sampler_name, num_trials=num_trials)
num_exp = num_trials / n * k

for count in packet_counts.itervalues():
    pct_dev = 100.0 * abs(num_exp - count) / num_exp
    print(pct_dev)

## 5.18 Compute the Spiral Ordering of a 2D array 

In [ ]:
def change_direction(increment):
    increment_new = increment[::-1]
    if abs(increment[0]) == 1:
        increment_new[1] *= -1
    return increment_new

def spiralize(arr):
    i_bounds = [0, len(arr) - 1]
    j_bounds = [0, len(arr[0]) - 1]
    increment = [0, 1]
    i = 0
    j = 0
    arr_spiral = []

    while i_bounds[1] >= i_bounds[0]:

        arr_spiral.append(arr[i][j])

        i_prop = i + increment[0]
        j_prop = j + increment[1]

        if j_prop > j_bounds[1]:
            i_bounds[0] += 1
        elif j_prop < j_bounds[0]:
            i_bounds[1] -= 1
        elif i_prop > i_bounds[1]:
            j_bounds[1] -= 1
        elif i_prop < i_bounds[0]:
            j_bounds[0] += 1
        else:
            i = i_prop
            j = j_prop
            continue

        increment = change_direction(increment)
        i += increment[0]
        j += increment[1]
        
    return arr_spiral

In [ ]:
arr = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]

print(np.array(arr))
print('')
print(spiralize(arr))

## 6.13: Find the first occurrence of a substring 

A good string search algorithm is fundamental to the performance of many applications. Several clever algorithms have been proposed for string search, each with its own trade-offs. As a result, there is no single perfect answer. If someone asks you this question in an interview, the best way to approach this problem would be to work through one good algorithm in detail and discuss at a high level other algorithms.

Given two strings s (the "search string") and t (the "text"), find the first occurrence of s in t.

Hint: Form a signature from a string

In [28]:
from collections import deque


def strings_are_same(str1, str2):
    for c1, c2 in zip(str1, str2):
        if c1 != c2:
            return False
    return True


class RollingHasher:
    """Following https://en.wikipedia.org/wiki/Rabin%E2%80%93Karp_algorithm"""
    
    def __init__(self, substring_len, base=256, prime_modulus=101):
        self.substring_len = substring_len
        self.base = base
        self.prime_modulus = prime_modulus
        self._char_q = deque()
        self._hash = 0
        
    def _add_to_hash(self, c):
        self._hash = (self._hash * self.base + ord(c)) % self.prime_modulus
        self._char_q.append(c)
        
    def _remove_first_in_from_hash(self):
        left_base_offset = 1
        for i in range(self.substring_len - 1):
            left_base_offset *= self.base 
            left_base_offset %= self.prime_modulus
        remove_val = ord(self._char_q.popleft())
        self._hash = self._hash + self.prime_modulus - remove_val * left_base_offset  # adding extra mod val ensures result isn't negative
        
    def get_hash(self, next_letter):
        if len(self._char_q) == self.substring_len:
            hash_sub = self._remove_first_in_from_hash()
        self._add_to_hash(next_letter)
        return self._hash if len(self._char_q) == self.substring_len else None


def find_substring_start_ind_simple(main_string, substring):
    substring_len = len(substring)
    for start_ind in range(0, len(main_string) - substring_len):
        if strings_are_same(substring, main_string[start_ind:start_ind + substring_len]):
            return start_ind
    return None


def find_substring_start_ind_set(main_string, substring):
    substring_hash = set([substring])
    substring_len = len(substring)
    for start_ind in range(0, len(main_string) - substring_len):
        if main_string[start_ind:start_ind + substring_len] in substring_hash:
            return start_ind
    return None

        
def find_substring_start_ind_rabin_karp(main_string, substring):
    sub_hasher = RollingHasher(len(substring))
    substring_hash = [sub_hasher.get_hash(c) for c in substring][-1]
    main_hasher = RollingHasher(len(substring))
    for i, c in enumerate(main_string):
        end_ind = i+1
        start_ind = end_ind - len(substring)
        if substring_hash == main_hasher.get_hash(c) and strings_are_same(substring, main_string[start_ind:end_ind]):
            return start_ind
    return None

In [29]:
main_string = 'gshgiuregbisnreipngseproingseonvdk;nfvdklldksfl'
substring = 'gbisn'

print(find_substring_start_ind_simple(main_string, substring))
print(find_substring_start_ind_set(main_string, substring))
print(find_substring_start_ind_rabin_karp(main_string, substring))

8
8
8


## 6.6: Reverse all the words in a sentence

Given a string containing a set of words separated by whitespace, we would like to transform it to a string in which the words appear in reverse order. For example, "Alice likes Bob" transforms to "Bob likes Alice". We do not need to keep the original string.

Implement a function for reversing the words in a string s.

Hint: It's difficult to solve this with one pass.

In [63]:
def reverse_words_simple(sentence):
    words = []
    word = ''
    for c in sentence:
        if c == ' ':
            words.append(word)
            words.append(' ')
            word = ''
        else:
            word += c
    words.append(word)
        
    sentence_rev = ''
    while len(words) > 0:
        sentence_rev += words.pop()
        
    return sentence_rev

def reverse_words_prealloc(sentence):
    end_ind = len(sentence)
    word = ''
    sentence_rev = [' '] * len(sentence)
    for c in sentence:
        if c == ' ':
            start_ind = end_ind-len(word)
            sentence_rev[start_ind:end_ind] = word
            end_ind = start_ind - 1
            word = ''
        else:
            word += c
    sentence_rev[end_ind-len(word):end_ind] = word
        
    return ''.join(sentence_rev)

In [64]:
sentence = 'Bob likes Alice'
print(reverse_words_simple(sentence))
print(reverse_words_prealloc(sentence))

Alice likes Bob
Alice likes Bob


## 6.7: Compute all mnemonics for a phone number 

Each digit, apart from 0 and 1, in a phone keypad corresponds to one of hte three or four letters of the alphabet, as shown in figure 6.1. Since words are easier to remember than number, it is natural to ask if a 7 or 10-digit phone number can be represented by a word. For examples, "2276696" corresponds to "ACRONYM" as well as "ABPOMZN",

Write a program which takes as input a phone number, specified as a string of digits, and returns all possible character sequences that correspond to the phone number. The cell phone keypad is specified by a mapping that takes a digit and returns the corresponding set of characters. The character sequences do not have to be legal words or phrases.

Hint: Use recursion.

In [75]:
DIGIT_LETTERS = {
    '2': ['A', 'B', 'C'],
    '3': ['D', 'E', 'F'],
    '4': ['G', 'H', 'I'],
    '5': ['J', 'K', 'L'],
    '6': ['M', 'N', 'O'],
    '7': ['P', 'Q', 'R', 'S'],
    '8': ['T', 'U', 'V'],
    '9': ['W', 'X', 'Y', 'Z']
}


def get_mnemonics(digits):
    if len(digits) > 1:
        prev_sequences = get_mnemonics(digits[:-1])
        digit = digits[-1]
        letters = DIGIT_LETTERS[digit]
        
        new_sequences = []
        for seq in prev_sequences:
            for letter in letters:
                new_sequences.append(seq + letter)
    else:
        new_sequences = DIGIT_LETTERS[digits]
        
    return new_sequences


digits = '23'
print(get_mnemonics(digits))

['AD', 'AE', 'AF', 'BD', 'BE', 'BF', 'CD', 'CE', 'CF']


In [68]:
set.

ab
b



In [77]:
set(['a' ,'b']).symmetric_difference(['x', 'y'])

{'a', 'b', 'x', 'y'}

In [ ]:
set.symmetric_difference